## **프로젝트: 멋진 작사가 만들기**

### **Step 1. 데이터 읽어오기**

In [1]:
# 필요한 라이브러리 import
import glob
import os, re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# 데이터 불러오기
txt_file_path = 'C:/Users/Minjoo Lee/AIFFEL/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담기
for txt_file in txt_list:
    with open(txt_file, 'rt', encoding='UTF8') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough"]


### **Step 2. 데이터 정제**

In [3]:
# 정제 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

# 문장이 어떻게 필터링 되는지 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 정제 데이터 구축
# 정제된 문장을 담을 리스트
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 원하지 않는 문장은 건너뛰기
    if len(sentence) == 0: continue # 길이가 0
    if sentence[-1] == ":": continue # 문장의 끝이 :
    
    # preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 확인
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

In [5]:
# 데이터를 토큰화하는 함수
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장 완성
    tokenizer.fit_on_texts(corpus)
    # tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 304  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2  23  77 ...   0   0   0]
 [  2  42  26 ...   0   0   0]
 [  2  23  77 ...   0   0   0]] <keras.preprocessing.text.Tokenizer object at 0x0000021701FE7788>


생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력해보자.

In [6]:
print(tensor[:3, :10])

[[   2  304   28   99 4811    3    0    0    0    0]
 [   2  221   13   85  226    6  115    3    0    0]
 [   2   24   17 1087 2347    3    0    0    0    0]]


In [7]:
# 단어 사전이 어떻게 구축되었는지 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


2번 인덱스가 `<start>`여서 모든 행이 2로 시작하는 것을 확인할 수 있다.

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  304   28   99 4811    3    0    0    0    0    0    0    0    0]
[ 304   28   99 4811    3    0    0    0    0    0    0    0    0    0]


행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩을 채워 넣은 것이다.

### **Step 3. 평가 데이터셋 분리**

In [9]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)

In [10]:
# 데이터셋 객체 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

### **Step 4. 인공지능 만들기**

In [11]:
# 모델 구현
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
    
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 512 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기
hidden_size = 2048 # 모델에 얼마나 많은 일꾼을 둘 것인가?
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) 

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break
    
# 불러온 데이터를 모델에 넣어보기
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.96383931e-04,  3.06807087e-05, -3.20026011e-04, ...,
          1.60722033e-04,  3.85537860e-04, -7.05594794e-05],
        [ 4.32939676e-04, -5.30474063e-05, -5.92583965e-04, ...,
          9.65326690e-05,  4.65600635e-04,  3.21498965e-06],
        [ 6.51538779e-04,  1.51049739e-04, -5.17855748e-04, ...,
          2.70104356e-04,  5.20643138e-04,  4.49553336e-05],
        ...,
        [-1.16748262e-04, -3.91560374e-04, -1.09337980e-03, ...,
          1.65157788e-03,  3.31841479e-03,  7.40238174e-04],
        [-1.83879092e-04, -6.69284374e-04, -1.39642623e-03, ...,
          1.94959925e-03,  3.65324854e-03,  1.10658945e-03],
        [-2.41904272e-04, -9.30300623e-04, -1.70498784e-03, ...,
          2.22853385e-03,  3.94132733e-03,  1.45016098e-03]],

       [[ 1.96383931e-04,  3.06807087e-05, -3.20026011e-04, ...,
          1.60722033e-04,  3.85537860e-04, -7.05594794e-05],
        [ 9.39750971e-05, -2.05085133e-04, -4

In [13]:
# 모델 구조 확인
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  6144512   
                                                                 
 lstm (LSTM)                 multiple                  20979712  
                                                                 
 lstm_1 (LSTM)               multiple                  33562624  
                                                                 
 dense (Dense)               multiple                  24590049  
                                                                 
Total params: 85,276,897
Trainable params: 85,276,897
Non-trainable params: 0
_________________________________________________________________


모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없다.

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 86s 123ms/step - loss: 3.4990
Epoch 2/10
686/686 [==============================] - 86s 126ms/step - loss: 2.9381
Epoch 3/10
686/686 [==============================] - 87s 126ms/step - loss: 2.6474
Epoch 4/10
686/686 [==============================] - 85s 124ms/step - loss: 2.3824
Epoch 5/10
686/686 [==============================] - 86s 125ms/step - loss: 2.1314
Epoch 6/10
686/686 [==============================] - 85s 124ms/step - loss: 1.8924
Epoch 7/10
686/686 [==============================] - 86s 125ms/step - loss: 1.6719
Epoch 8/10
686/686 [==============================] - 85s 123ms/step - loss: 1.4775
Epoch 9/10
686/686 [==============================] - 85s 123ms/step - loss: 1.3135
Epoch 10/10
686/686 [==============================] - 86s 125ms/step - loss: 1.1811


### **Step 5. 평가하기**

In [15]:
#문장 생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장 만들기
    while True: # 루프를 돌면서 init_sentence에 단어를 하나씩 생성
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated # 최종적으로 모델이 생성한 문장을 반환

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love the way you shake your thing <end> '

---